In [26]:
!pip install numpy==1.24.4


In [27]:
import gdown

file_id = "1klN3WXILu4YVvVMHIl6h_fryzJ0pD73R"
url = f"https://drive.google.com/uc?id={file_id}"
output = "train.csv"


gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1klN3WXILu4YVvVMHIl6h_fryzJ0pD73R
To: /content/train.csv
100%|██████████| 63.3M/63.3M [00:00<00:00, 66.7MB/s]


'train.csv'

In [28]:
import pandas as pd
import torch
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'gpu')
print(f'Using device: {device}')

# Load the dataset
data = pd.read_csv('evaluation.csv', delimiter=';', engine='python', on_bad_lines='skip').head(10000)
data2 = pd.read_csv('test.csv', delimiter=';', encoding='cp1252', engine='python', on_bad_lines='skip').head(10000)


data = data.drop(columns=['Unnamed: 0'])
data2 = data2.drop(columns=['Unnamed: 0'])


Using device: cuda


In [29]:
data.head()

,title,text,label
0,"Sanders back in U.S. Senate, blasts 'coloniali...",WASHINGTON (Reuters) - Democratic U.S. preside...,1
1,Kremlin: Syria peoples' congress being 'active...,MOSCOW (Reuters) - A proposal to convene a con...,1
2,Oregon Cop Convicted Of Shattering Biker’s Co...,"In a baffling fit of rage, an Oregon State Pol...",0
3,Twitter Erupts With Glee Over #CruzSexScandal...,The last thing any politician running for the ...,0
4,MUST WATCH VIDEO: Obama Tries To Trash Trump B...,This is too good to miss! Mr. Teleprompter did...,0


In [6]:
pip install transformers datasets scikit-learn torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [30]:
# Step 1: Combine title and text
data['content'] = data['title'].fillna('') + ' ' + data['text'].fillna('')
data2['content'] = data2['title'].fillna('') + ' ' + data2['text'].fillna('')

# Keep only necessary columns
data = data[['content', 'label']]
data2 = data2[['content', 'label']]


In [31]:
# Step 2: Convert to Hugging Face Datasets
from datasets import Dataset
train_dataset = Dataset.from_pandas(data)
test_dataset = Dataset.from_pandas(data2)


In [32]:
# Step 3: Load DistilBERT tokenizer
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def tokenize_function(example):
    return tokenizer(example['content'], padding="max_length", truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)



Map:   0%|          | 0/8117 [00:00<?, ? examples/s]

Map:   0%|          | 0/8117 [00:00<?, ? examples/s]

In [33]:
# No need to rename 'label' to 'labels' — it already exists
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


ValueError: Columns ['labels'] not in the dataset. Current columns in the dataset: ['content', 'label', 'input_ids', 'attention_mask']

In [34]:
# Step 5: Load DistilBERT for classification
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.to(device)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [35]:
# Step 6: Define training arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
        num_train_epochs=3,
            per_device_train_batch_size=16,
                per_device_eval_batch_size=16,
                    learning_rate=2e-5,
                        logging_dir="./logs",
                            logging_steps=10
                            )


In [36]:
# Step 7: Define metrics
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
   }


In [37]:
# Step 8: Initialize Trainer and train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    )

trainer.train()


Step,Training Loss
10,0.649100
20,0.436200
30,0.246100
40,0.077100
50,0.091600
60,0.069800
70,0.076200
80,0.107700
90,0.069300
100,0.040000


Step,Training Loss
10,0.649100
20,0.436200
30,0.246100
40,0.077100
50,0.091600
60,0.069800
70,0.076200
80,0.107700
90,0.069300
100,0.040000


TrainOutput(global_step=1524, training_loss=0.03980493376302237, metrics={'train_runtime': 1152.5504, 'train_samples_per_second': 21.128, 'train_steps_per_second': 1.322, 'total_flos': 3225713624672256.0, 'train_loss': 0.03980493376302237, 'epoch': 3.0})

In [38]:
# Step 9: Evaluate model
trainer.evaluate()


{'eval_loss': 0.05969441309571266,
 'eval_accuracy': 0.9852162128865344,
 'eval_f1': 0.9862668802929733,
 'eval_precision': 0.9851394604481024,
 'eval_recall': 0.9873968835930339,
 'eval_runtime': 116.2457,
 'eval_samples_per_second': 69.826,
 'eval_steps_per_second': 4.37,
 'epoch': 3.0}

In [40]:
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)

from sklearn.metrics import classification_report
print(classification_report(test_dataset["label"], preds))


              precision    recall  f1-score   support

           0       0.99      0.98      0.98      3753
           1       0.99      0.99      0.99      4364

    accuracy                           0.99      8117
   macro avg       0.99      0.99      0.99      8117
weighted avg       0.99      0.99      0.99      8117



In [41]:
trainer.save_model("saved_model")
tokenizer.save_pretrained("saved_model")


('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.txt',
 'saved_model/added_tokens.json',
 'saved_model/tokenizer.json')

In [43]:
from google.colab import files
files.download('saved_model')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>